In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('/kaggle/input/mlub-learning-to-count/train.txt',sep=' ',header=None)

# take only the first 200 images
#df_train = df_train.head(500)

# Estructura del dataset

In [ ]:
df_train

In [ ]:
df_train.describe()

In [ ]:
import imageio
from skimage import transform,io
import matplotlib.pyplot as plt
# read images and store into a np array

data_dir = '/kaggle/input/mlub-learning-to-count/train/'
im_size = 128
N = df_train.shape[0]
X = np.zeros((N, im_size,im_size))
y = np.zeros((N))
cont =0

for ind, item in df_train.iterrows():
    im       = imageio.imread(data_dir + item[0])/255. 
    small_im = transform.resize(im, (im_size,im_size), mode='symmetric', preserve_range=True)
    X[cont, :,:] = small_im
    y[cont] = item[1]
    cont+=1
    
plt.imshow(small_im,cmap='gray')
plt.show()

In [ ]:
im = imageio.imread(data_dir + item[0])
num = item[1]
print("Tamaño de la imagen: " + str(im.shape))
print("Etiqueta num de personas: " + str(num))


plt.imshow(im)
plt.show()

# Aumentacion de las imágenes

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range = 15,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = "nearest")

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val= train_test_split(X, y, test_size=0.33, random_state=66)

X_train = X_train.reshape(X_train.shape + (1,))
X_val = X_val.reshape(X_val.shape + (1,))

In [ ]:
from sklearn.metrics import mean_squared_error
"""
plt.plot(rfc_predict,y_val,'.')

print(np.sqrt(mean_squared_error(rfc_predict,y_val)))
"""

In [ ]:
## With a Neural Network

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout, Flatten, Conv2D, MaxPool2D, Dense


keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
model = keras.models.Sequential()
model.add(Conv2D(filters=32, kernel_size=5, padding="same", activation="relu", input_shape=[128, 128, 1]))
model.add(Conv2D(filters=32, kernel_size=5, padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=2, strides=2, padding='valid'))

model.add(Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=2, strides=2, padding='valid'))

model.add(Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=2, strides=2, padding='valid'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(keras.layers.Dense(1, activation="linear"))

In [ ]:
model.layers

In [ ]:
model.summary()


In [ ]:
#model = keras.models.load_model('/kaggle/working/model_counts.hdf5')

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999, amsgrad = True)
model.compile(loss="mean_squared_error", optimizer="Adam",metrics=["mean_squared_error",tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
from keras.callbacks import ReduceLROnPlateau
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 75)

# Guardamos el mejor modelo con menor error de validación 
checkpointer = ModelCheckpoint(filepath = "model_counts.hdf5", verbose = 1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', mode = 'auto', factor=0.2, verbose = 1, patience=15, min_lr=0.0001)

In [ ]:
history = model.fit(train_datagen.flow(X_train, y_train, batch_size=64),steps_per_epoch=len(X_train) // 64, 
                    epochs=500, validation_data=(X_val, y_val),
                    verbose=1,callbacks=[earlystopping, checkpointer,reduce_lr])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model_json = model.to_json()
with open("model.json","w") as json_file:
    json_file.write(model_json)

In [ ]:
#model = keras.models.load_model('/kaggle/working/model_counts.hdf5')

In [ ]:
import pandas as pd

pd.DataFrame(history.history)[['loss','val_loss']].plot(figsize=(8, 5))
plt.grid(True)
#plt.gca().set_ylim(0.4, 0.8)
plt.show()

In [ ]:
y_model1 = model.predict(X_val)

plt.plot(y_model1,y_val,'.')
np.sqrt(mean_squared_error(y_model1,y_val))

In [ ]:
## evaluate test and generate submission
test_dir = '/kaggle/input/mlub-learning-to-count/test/test/'
im_size = 128
N = 500
X_test = np.zeros((N, im_size,im_size))
cont =0

for x in range(500):
    im       = imageio.imread(test_dir + 'test_composite'+str(x).zfill(9) + '.png')/255.
    small_im = transform.resize(im, (im_size,im_size), mode='symmetric', preserve_range=True)
    X_test[cont, :,:] = small_im
    cont+=1
    
plt.imshow(small_im)
plt.show()

In [ ]:
X_test = X_test.reshape(X_test.shape + (1,))

# create the file to make the sumbission

y_test = model.predict(X_test)
y_test = [int(x[0]) for x in y_test]

df_output = pd.DataFrame(y_test)
df_output.index.name = 'index'
df_output.columns = ['prediction']
df_output.to_csv('output.csv')
